<a href="https://colab.research.google.com/github/advik-2402/HumanEmotionRecognition/blob/main/Polygence_Project_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/misbah4064/facial_expressions.git

In [ ]:
%cd facial_expressions/
%mkdir -p data_set/{happy,anger,surprise,sad,neutral}

In [ ]:
import cv2 

emotions = ['happy', 'anger', 'surprise', 'sad','neutral']

for emotion in emotions:
    with open(f'{emotion}.txt','r') as f:
        img = [line.strip() for line in f] #storing "spaceless" lines of text in img variable

    for image in img:
        loadedImage = cv2.imread(f"images/{image}") #loads images from the images folder
        cv2.imwrite(f"data_set/{emotion}/{image}",loadedImage)  #saves images in the "data_set" folder
    print(f"done writing {emotion} images")

In [ ]:
%mkdir dataset

In [ ]:
import cv2

# updated code to automate giving id to emotion
emotions = {'happy': 0, 'anger': 1, 'surprise': 2, 'sad': 3, 'neutral': 4}

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

for emotion in emotions:
  with open(emotion + '.txt', 'r') as f:
    images = [line.strip() for line in f]

    face_id = emotions[emotion]

    count = 0
    
    for image in images:
      img = cv2.imread("data_set/" + emotion + "/" + image) #reads the images one by one
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #the image is converted to grayscale
      faces = face_detector.detectMultiScale(gray, 1.3, 5) #detects faces in greyscale using pre-trained Haar Cascade
      
      #output is a list of rectangles that represent the bounding boxes of the detected faces in the image.
      
      for (x,y,w,h) in faces: #(x,y) is top left corner of the rect, w is width, h is height

        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2) #drawing a rect around the detected face. thickness 2, color blue.
        count += 1 #count of faces found

        # Saves the captured face data into dataset folder with face id and count.
        cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", gray[y:y+h,x:x+w])

print("\n Done creating face data", emotion)

In [ ]:
%mkdir trainer

In [ ]:
import cv2 
import numpy as np
from PIL import Image
import os

# Path for face image database
path = 'dataset'

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# function to get the images and label data from the specified path
def getImagesAndLabels(emotion_path, emotion_id):

    imagePaths = [os.path.join(emotion_path,f) for f in os.listdir(emotion_path)]     
    faceSamples=[]
    ids = []

    #The function loops over all images in the folder, detects faces using the cascade classifier, 
    #extracts the face region, and appends it to the face samples list along with the emotion ID.
    for imagePath in imagePaths:
        loadedImage = cv2.imread(imagePath)
        gray = cv2.cvtColor(loadedImage, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        for (x,y,w,h) in faces:
            faceSamples.append(gray[y:y+h,x:x+w])
            ids.append(emotion_id)

    return faceSamples,ids

# Train the model on all emotions
emotions = {
    'happy': 0,
    'anger': 1,
    'surprise': 2,
    'sad': 3,
    'neutral': 4,
}

print ("\n [INFO] Training faces....")

# The all_faces and all_ids lists are used to accumulate the face samples and IDs for all emotions, 
#and these lists are passed to the train method of the recognizer to train the model on all emotions.

all_faces = []
all_ids = []

for emotion, emotion_id in emotions.items():
    emotion_path = f"data_set/{emotion}/"
    faces, ids = getImagesAndLabels(emotion_path, emotion_id)
    all_faces.extend(faces)
    all_ids.extend(ids)

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(all_faces, np.array(all_ids))

# The model is saved in a yml file
recognizer.write('trainer/trainer.yml') 

# The number of emotions trained is printed and program is exited
print("\n [INFO] {0} Emotions trained. Exiting Program".format(len(np.unique(all_ids))))

In [ ]:
import cv2
import numpy as np
import os 

# Load the trained model
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainer/trainer.yml')

# Load the cascade classifier
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);

font = cv2.FONT_HERSHEY_SIMPLEX

# Emotions related to ids: example ==> Happy: id=0,  etc
names = ['Happy', 'Anger', 'Sad', 'Surprise', 'Neutral'] 

# Load the image
img = cv2.imread("elon.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = faceCascade.detectMultiScale( 
    gray,
    scaleFactor = 1.2,
    minNeighbors = 5,
    )

# Loop through each detected face
for (x,y,w,h) in faces:

    # Draw a rectangle around the face
    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

    # Predict the emotion for the face
    id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

    # Check if confidence is less than 100
    if (confidence < 100):
        id = names[id]
        confidence = "  {0}%".format(round(100 - confidence))
    else:
        id = "unknown"
        confidence = "  {0}%".format(round(100 - confidence))
    
    # Put the predicted emotion and confidence score on the image
    cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
    cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  

# Save the image with the predicted emotions and confidence scores
cv2.imwrite("elon.jpg",img) 

print("\n [INFO] Done detecting and Image is saved")

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

image = cv2.imread("elon.jpg")
height, width = image.shape[:2]
resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

fig, ax = plt.subplots(figsize=(18, 10))
ax.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
ax.axis("off")
plt.show()

In [ ]:
from google.colab import files
file = files.upload()

In [ ]:
import pandas as pd

music_df = pd.read_csv("data_moods.csv") #reading dataset through pandas

music_df = music_df[['name','artist','mood','popularity']]
#music_df.head() #displaying the above columns of the dataset

In [ ]:
# Making Songs Recommendations Based on Predicted Class
def Recommend_Songs(id):
    
    if( id =='Happy' ):

        Play = music_df[music_df['mood'] =='Happy']
        Play = Play.sort_values(by="popularity", ascending=False)
        Play = Play[:5].reset_index(drop=True)
        display(Play)

    elif( id =='Anger' or id =='Neutral' ):

        Play = music_df[music_df['mood'] =='Calm' ]
        Play = Play.sort_values(by="popularity", ascending=False)
        Play = Play[:5].reset_index(drop=True)
        display(Play)

    elif( id =='Sad' ):

        Play = music_df[music_df['mood'] =='Sad' ]
        Play = Play.sort_values(by="popularity", ascending=False)
        Play = Play[:5].reset_index(drop=True)
        display(Play)
        
    elif( id =='Surprise'):

        Play = music_df[music_df['mood'] =='Energetic' ]
        Play = Play.sort_values(by="popularity", ascending=False)
        Play = Play[:5].reset_index(drop=True)
        display(Play)

Recommend_Songs(id)
